In [1]:
import pandas as pd
import numpy as np 
import pickle
import matplotlib.pyplot as plt

In [2]:
model_num = 1
x_train, x_test, y_train, y_test = pickle.load(open(f'Pickles/TTS_{model_num}.p', 'rb'))
y_train = y_train.Target.values.ravel() 
y_test = y_test.Target.values.ravel()

In [3]:
x_train.head()

,Volume,DayDiff,ab,abbvie,ability,able,abroad,abu,accelerate,accept,...,yield,yields,york,young,yuan,zealand,zero,zone,zte,zuckerberg
0,-0.683618,0.504059,0,0,0,1,0,0,1,0,...,0,0,1,1,2,0,0,0,0,0
1,2.175385,3.872091,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.232363,-0.844652,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-0.406931,0.106939,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,2
4,-0.629237,0.301753,0,0,0,0,1,0,0,0,...,0,0,0,0,2,0,0,0,0,0


## Sklearn

In [4]:
from sklearn.linear_model import SGDClassifier, RidgeClassifier, LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC,LinearSVC
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, plot_confusion_matrix, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from tqdm import tqdm
import Functions as func

In [ ]:
models = {'Log': LogisticRegression(), 'Knn': KNeighborsClassifier(), 'DT': DecisionTreeClassifier(random_state = 10), 
          'Gaussian': GaussianNB(), 'LDA': LinearDiscriminantAnalysis(),
          'LinearSVC': LinearSVC(max_iter = 1250, random_state = 10), 'SDGSVC': SGDClassifier(random_state = 10),  
          'ADA': AdaBoostClassifier(random_state = 10), 'Bagging': BaggingClassifier(random_state = 10), 
          'Ridge': RidgeClassifier(random_state = 10), 'RF': RandomForestClassifier(random_state = 10)}

#create stacked model
new_models = func.stacked_model(models)

# getting results and model
result_dict = func.test_models(x_train, y_train, new_models, n_jobs = 12)
func.plot_model_results(result_dict, filepath = 'Images/Sklearn/Stacked1.png')

Evaluating STACKED:  92%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎          | 11/12 [01:11<00:06,  6.75s/it]

In [ ]:
models = {'Log': LogisticRegression(), 'DT': DecisionTreeClassifier(random_state = 10), 
          'Gaussian': GaussianNB(), 'LinearSVC': LinearSVC(max_iter = 1250, random_state = 10), 
          'ADA': AdaBoostClassifier(random_state = 10), 'Bagging': BaggingClassifier(random_state = 10), 
          'RF': RandomForestClassifier(random_state = 10)}

#create stacked model
new_models = func.stacked_model(models)

# getting results and model
result_dict = func.test_models(x_train, y_train, new_models, n_jobs = 12)
func.plot_model_results(result_dict, filepath = 'Images/Sklearn/Stacked2.png')

In [ ]:
# rf = RandomForestClassifier(random_state = 10, n_jobs = 10)
# rf.fit(x_train, y_train)
# rf.score(x_test, y_test)

# lg = LogisticRegression(random_state = 10, n_jobs = 10)
# lg.fit(x_train, y_train)
# lg.score(x_test, y_test)

## Keras